In [1]:
from pathlib import Path
wd = Path.cwd()
wd = wd.parent.parent
wd = wd / 'merged_data' 
sub1 = str(wd) + '/subtask1.parquet'
print(sub1)

/home/matijak/Documents/programiranje/projects/semeval/merged_data/subtask1.parquet


In [2]:
import pandas as pd
df = pd.read_parquet(sub1)
#df.head(n=20)
df.aggregate

<bound method DataFrame.aggregate of      lang    art_name             entity start   end       class1  \
0      BG  BG_670.txt              Запад   152   156   Antagonist   
1      BG  BG_670.txt                САЩ   530   532   Antagonist   
2      BG  BG_670.txt               НАТО   535   538   Antagonist   
3      BG  BG_670.txt            Украйна   578   584   Antagonist   
4      BG  BG_670.txt  украински войници   633   649     Innocent   
...   ...         ...                ...   ...   ...          ...   
2530   PT  PT_159.txt              China   371   375     Innocent   
2531   PT   PT_74.txt           Pokrovsk  1139  1146     Innocent   
2532   PT   PT_31.txt            Ucrânia   313   319   Antagonist   
2533   PT   PT_31.txt                EUA   327   329   Antagonist   
2534   PT   PT_31.txt            Moscovo  1999  2005  Protagonist   

                                          classes2  \
0     [Conspirator, Instigator, Foreign Adversary]   
1                         

In [3]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text

df['label'] = df.apply(labelNum,axis=1)
#print(df['label'].value_counts(),
#df['class1'].value_counts())
#df['input'] = df['text']
df['input'] = df.apply(cleanText,axis=1)
#print(df['input'].iloc[0])

In [4]:
data = df.loc[ : , ['input','label']]
data.aggregate

<bound method DataFrame.aggregate of                                                   input  label
0     Опитът на колективния Запад да „обезкърви Руси...      0
1     Опитът на колективния Запад да „обезкърви Руси...      0
2     Опитът на колективния Запад да „обезкърви Руси...      0
3     Опитът на колективния Запад да „обезкърви Руси...      0
4     Опитът на колективния Запад да „обезкърви Руси...      1
...                                                 ...    ...
2530  A transição energética Multiplicam-se os fenóm...      1
2531  Rússia assume controlo de mais uma povoação no...      1
2532  Quais foram as consequências do ataque de Iska...      0
2533  Quais foram as consequências do ataque de Iska...      0
2534  Quais foram as consequências do ataque de Iska...      2

[2535 rows x 2 columns]>

In [5]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).to(device)
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def preprocess_function(examples):
    return tokenizer(examples['input'], padding=True, truncation=True, max_length=1024, return_offsets_mapping=True)
df_en = df[ df['lang'] == 'EN']
test_row = df_en.iloc[0]
print(test_row)

Using device: cuda


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


lang                                                       EN
art_name                                     EN_UA_103861.txt
entity                                                Chinese
start                                                     791
end                                                       797
class1                                             Antagonist
classes2                                                [Spy]
text        The World Needs Peacemaker Trump Again \n\n by...
label                                                       0
input       The World Needs Peacemaker Trump Again  by Jef...
Name: 448, dtype: object


In [6]:
import re

def find_all_substring_start_end(text, substring):
    # Use re.finditer to find all occurrences of the substring in the text
    matches = re.finditer(re.escape(substring), text)
    
    # Collect the start and end indices of all matches
    positions = [(match.start(), match.end()) for match in matches]
    
    return positions
def adjust_start_end(org_text,cl_text,start,end,entity):
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    i = ss1.index((start,end+1))
    return ss2[i][0],ss2[i][1]

astart,aend = adjust_start_end(str(test_row['text']),str(test_row['input']),int(test_row['start']),int(test_row['end']),test_row['entity'])
print(astart,aend,int(test_row['start']),int(test_row['end'])  + 1)
print(test_row['text'][ int(test_row['start'])  : int(test_row['end'])  + 1])
print(test_row['input'][astart : aend])

785 792 791 798
Chinese
Chinese


In [7]:
st = preprocess_function(test_row)

In [8]:

#print(tokenizer.convert_ids_to_tokens(ids = st['input_ids']))
offsets = st['offset_mapping']
#print(offsets)
#print(test_row['input'][offsets[7][0] : offsets[7][1]])
#print(test_row['start'],test_row['end'],test_row['entity'],test_row['text'][791:797+1])
#print(test_row['input'])
i2 =offsets.index((astart,aend))
print(i2)
ID = st['input_ids'][i2]
print(i2,ID)
#print(offsets[i2])

180
180 76438


In [9]:
input_text = test_row['input']

def preprocess_function2(examples):
    return tokenizer(examples, padding=True, truncation=True, return_tensors='pt',max_length=2048)
    
inputs = preprocess_function2(input_text)

def slices(index,size):
    print(size)
    if index < 255:
        return 0,512
    elif index >= 255:
        if index + 256 > size:
            return index-(512-(size-index)), size
        else:
            return index-255,index+256+1

def slices2(index,size,context_size):
    lower_c = int(context_size/2-1)
    upper_c = int(context_size/2)
    print(lower_c,upper_c)
    if index < lower_c:
        return 0,context_size
    elif index >= lower_c:
        if index + upper_c > size:
            return index-(context_size-(size-index)), size
        else:
            return index-lower_c,index+upper_c+1  

supper,slower = slices(i2,len(inputs['input_ids'][0]))
supper2,slower2 = slices2(i2,len(inputs['input_ids'][0]),512)
print(supper,slower,supper2,slower2)

new_id = inputs['input_ids'][0][supper2:slower2].to(device)
new_id2 = new_id.reshape(1,-1)
new_att = inputs['attention_mask'][0][supper2:slower2].to(device)
new_att2 = new_att.reshape(1,-1)
niputs = {
    "input_ids" : new_id2,
    "attention_mask" : new_att2
}
i3 = new_id.tolist().index(ID)
print(i3,new_id[i3])
#print(inputs['input_ids'][0][supper:slower].to(device),len(inputs2),supper,slower)
inputs.to(device)
#print(inputs)
#print(niputs)
with torch.no_grad():  # Disable gradients for inference
    outputs = model(**niputs,output_hidden_states=True)
hidden_states = outputs.hidden_states
last_hs = hidden_states[-1]
print(last_hs.shape,last_hs[0][i3])

1117
255 256
0 512 0 512
99 tensor(76438, device='cuda:0')
torch.Size([1, 512, 768]) tensor([-5.8830e-02,  3.4325e-02, -3.6241e-02,  6.7208e-02,  1.7106e-01,
        -4.5735e-02, -6.1728e-02,  2.5603e-01,  2.5663e-02,  2.9910e-02,
         3.4733e-01, -5.1959e-03,  4.4397e-02, -5.9194e-02, -3.2795e-02,
         5.5843e-02,  6.0932e-02, -6.3790e-02,  1.7835e-01,  5.1217e-02,
        -6.0373e-02,  8.4304e-03, -1.9966e-02,  5.0501e-02, -3.4988e-02,
         7.5383e-02, -3.8825e-02, -1.0471e-01, -2.8355e-02, -5.2300e-03,
        -7.3352e-03, -9.5749e-03,  6.7404e-02,  1.1210e-02, -7.3397e-03,
         1.3537e-01, -1.8678e-02,  3.3205e-02, -5.7537e-02,  7.7814e-02,
         3.2125e-02,  9.8975e-03, -9.4488e-02,  3.7887e-02,  5.3757e-02,
        -3.3313e-02,  1.7805e-02,  7.0484e-02,  3.7347e-02, -8.7805e-02,
        -2.9893e-02,  2.2748e-01,  7.7393e-02,  2.0137e-01,  1.8671e-01,
        -4.6239e-02,  3.9719e-02, -3.0399e-02,  5.3178e-02,  7.7169e-02,
         8.2557e-02,  2.1522e-03, -3.72